In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from mlxtend.evaluate import PredefinedHoldoutSplit
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [3]:
X_full = pd.read_csv('New_Train_Features_Full.csv')
X_test_full = pd.read_csv('New_Test_Features_Full.csv')
y = pd.read_csv('Train_Target.csv').iloc[:,1]

X_full_no = pd.read_csv('New_Train_Features.csv')
X_test_full_no = pd.read_csv('New_Test_Features.csv')

X_no_manual = pd.read_csv('New_Train_Features_No_Selected.csv')
X_test_no_manual = pd.read_csv('New_Test_Features_No_Selected.csv')

X_RFE = pd.read_csv('New_Train_Features_RFE.csv')
X_RFE_test = pd.read_csv('New_Test_Features_RFE.csv')

X_LASSO = pd.read_csv('New_Train_Features_LASSO.csv')
X_LASSO_test = pd.read_csv('New_Test_Features_LASSO.csv')

X_manual = pd.read_csv('New_Train_Features_Selected.csv')
X_manual_test = pd.read_csv('New_Test_Features_Selected.csv')

# Full Dataset:

In [4]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [5]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [17]:
params = {
    'estimator': [
        DecisionTreeClassifier(max_depth = 1, random_state = 2025),
        DecisionTreeClassifier(max_depth = 2, random_state = 2025)],
    'n_estimators': [10, 20, 50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0]
}

ada = AdaBoostClassifier(random_state = 2025)

In [18]:
np.random.seed(2025)
ada_grid = GridSearchCV(ada, param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
ada_grid.fit(X_temp, y_temp)
print('Best Parameters:', ada_grid.best_params_)

Best Parameters: {'estimator': DecisionTreeClassifier(max_depth=2, random_state=2025), 'learning_rate': 0.5, 'n_estimators': 200}


In [19]:
np.random.seed(2025)
ada_model = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 2, random_state = 2025), learning_rate = 0.5, n_estimators = 200, random_state = 2025)
ada_model.fit(X_temp, y_temp)
y_pred = ada_model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [20]:
np.random.seed(2025)
ada_model.fit(X_full, y)
y_pred_ada = ada_model.predict(X_test_full)

In [21]:
from google.colab import files

pred_ada = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_ada})

with open('ADA_Full_Cov.csv', 'w') as file:
    pred_ada.to_csv(file, index = False, header = True)

files.download('ADA_Full_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# RFE Dataset:

In [22]:
np.random.seed(2025)
X_train_rfe, X_temp_rfe, y_train_rfe, y_temp_rfe = train_test_split(X_RFE, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_rfe, X_test_rfe, y_val_rfe, y_test_rfe = train_test_split(X_temp_rfe, y_temp_rfe, test_size = 0.5, random_state = 2025, stratify = y_temp_rfe)

In [23]:
np.random.seed(2025)
train_rfe_ind, val_rfe_ind = train_test_split(np.arange(X_temp_rfe.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_rfe)
split_rfe = PredefinedHoldoutSplit(valid_indices = val_rfe_ind)

In [24]:
np.random.seed(2025)
ada_grid_rfe = GridSearchCV(AdaBoostClassifier(random_state = 2025), param_grid = params, cv = split_rfe, n_jobs = -1, scoring = 'f1_macro')
ada_grid_rfe.fit(X_temp_rfe, y_temp_rfe)
print('Best Parameters:', ada_grid_rfe.best_params_)

Best Parameters: {'estimator': DecisionTreeClassifier(max_depth=2, random_state=2025), 'learning_rate': 1.0, 'n_estimators': 10}


In [25]:
np.random.seed(2025)
ada_model_rfe = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 2, random_state = 2025), learning_rate = 1.0, n_estimators = 10, random_state = 2025)
ada_model_rfe.fit(X_temp_rfe, y_temp_rfe)
y_pred_rfe = ada_model_rfe.predict(X_test_rfe)

print(classification_report(y_test, y_pred_rfe))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [26]:
np.random.seed(2025)
ada_model_rfe.fit(X_RFE, y)
y_pred_rfe_ada = ada_model_rfe.predict(X_RFE_test)

In [27]:
from google.colab import files

pred_ada = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_rfe_ada})

with open('ADA_RFE_Cov.csv', 'w') as file:
    pred_ada.to_csv(file, index = False, header = True)

files.download('ADA_RFE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LASSO:

In [28]:
np.random.seed(2025)
X_train_lasso, X_temp_lasso, y_train_lasso, y_temp_lasso = train_test_split(X_LASSO, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_lasso, X_test_lasso, y_val_lasso, y_test_lasso = train_test_split(X_temp_lasso, y_temp_lasso, test_size = 0.5, random_state = 2025, stratify = y_temp_lasso)

In [29]:
np.random.seed(2025)
train_lasso_ind, val_lasso_ind = train_test_split(np.arange(X_temp_lasso.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_lasso)
split_lasso = PredefinedHoldoutSplit(valid_indices = val_lasso_ind)

In [30]:
np.random.seed(2025)
ada_grid_lasso = GridSearchCV(AdaBoostClassifier(random_state = 2025), param_grid = params, cv = split_lasso, n_jobs = -1, scoring = 'f1_macro')
ada_grid_lasso.fit(X_temp_lasso, y_temp_lasso)
print('Best Parameters:', ada_grid_lasso.best_params_)

Best Parameters: {'estimator': DecisionTreeClassifier(max_depth=2, random_state=2025), 'learning_rate': 1.0, 'n_estimators': 50}


In [31]:
np.random.seed(2025)
ada_model_lasso = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 2, random_state = 2025), learning_rate = 1.0, n_estimators = 50, random_state = 2025)
ada_model_lasso.fit(X_temp_lasso, y_temp_lasso)
y_pred_lasso = ada_model_lasso.predict(X_test_lasso)

print(classification_report(y_test, y_pred_lasso))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [32]:
np.random.seed(2025)
ada_model_lasso.fit(X_LASSO, y)
y_pred_lasso_ada = ada_model_lasso.predict(X_LASSO_test)

In [33]:
from google.colab import files

pred_ada = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_lasso_ada})

with open('ADA_LASSO_Cov.csv', 'w') as file:
    pred_ada.to_csv(file, index = False, header = True)

files.download('ADA_LASSO_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization




In [34]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full_no, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [35]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [36]:
np.random.seed(2025)
ada_grid = GridSearchCV(AdaBoostClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
ada_grid.fit(X_temp, y_temp)
print('Best Parameters:', ada_grid.best_params_)

Best Parameters: {'estimator': DecisionTreeClassifier(max_depth=2, random_state=2025), 'learning_rate': 1.0, 'n_estimators': 10}


In [37]:
np.random.seed(2025)
ada_model = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 2, random_state = 2025), learning_rate = 1.0, n_estimators = 10, random_state = 2025)
ada_model.fit(X_temp, y_temp)
y_pred_ada_no = ada_model.predict(X_test)

print(classification_report(y_test, y_pred_ada_no))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [38]:
np.random.seed(2025)
ada_model.fit(X_full_no, y)
y_pred_ada_no = ada_model.predict(X_test_full_no)

In [39]:
from google.colab import files

pred_ada = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_ada_no})

with open('ADA_Full_No_Cov.csv', 'w') as file:
    pred_ada.to_csv(file, index = False, header = True)

files.download('ADA_Full_No_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization - Manual



In [40]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_no_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [41]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [42]:
np.random.seed(2025)
ada_grid = GridSearchCV(AdaBoostClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
ada_grid.fit(X_temp, y_temp)
print('Best Parameters:', ada_grid.best_params_)

Best Parameters: {'estimator': DecisionTreeClassifier(max_depth=2, random_state=2025), 'learning_rate': 0.5, 'n_estimators': 200}


In [43]:
np.random.seed(2025)
ada_model = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 2, random_state = 2025), learning_rate = 0.5, n_estimators = 200, random_state = 2025)
ada_model.fit(X_temp, y_temp)
y_pred_manual = ada_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [44]:
np.random.seed(2025)
ada_model.fit(X_no_manual, y)
y_pred_ada_no_manual = ada_model.predict(X_test_no_manual)

In [45]:
from google.colab import files

pred_ada = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_ada_no_manual})

with open('ADA_No_Manual_Cov.csv', 'w') as file:
    pred_ada.to_csv(file, index = False, header = True)

files.download('ADA_No_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Manual Selection:

In [46]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [47]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [48]:
np.random.seed(2025)
ada_grid = GridSearchCV(AdaBoostClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
ada_grid.fit(X_temp, y_temp)
print('Best Parameters:', ada_grid.best_params_)

Best Parameters: {'estimator': DecisionTreeClassifier(max_depth=2, random_state=2025), 'learning_rate': 1.0, 'n_estimators': 200}


In [49]:
np.random.seed(2025)
ada_model = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 2, random_state = 2025), learning_rate = 1.0, n_estimators = 200, random_state = 2025)
ada_model.fit(X_temp, y_temp)
y_pred_manual = ada_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [50]:
np.random.seed(2025)
ada_model.fit(X_manual, y)
y_pred_ada_manual = ada_model.predict(X_manual_test)

In [51]:
from google.colab import files

pred_ada = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_ada_manual})

with open('ADA_Manual_Cov.csv', 'w') as file:
    pred_ada.to_csv(file, index = False, header = True)

files.download('ADA_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# SMOTE FOR BEST ADA:

In [52]:
from imblearn.over_sampling import SMOTE

In [61]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full_no, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

smote = SMOTE(random_state = 2025)
X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

In [62]:
np.random.seed(2025)
ada_model_smote = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 2, random_state = 2025), learning_rate = 1.0, n_estimators = 10, random_state = 2025)
ada_model_smote.fit(X_resampled, y_resampled)
y_pred_smote = ada_model_smote.predict(X_test)

print(classification_report(y_test, y_pred_smote))
y_pred_smote_ada = ada_model_smote.predict(X_test_full_no)

              precision    recall  f1-score   support

           0       1.00      0.97      0.99        36
           1       0.90      1.00      0.95         9

    accuracy                           0.98        45
   macro avg       0.95      0.99      0.97        45
weighted avg       0.98      0.98      0.98        45



In [63]:
from google.colab import files

pred_ada = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_smote_ada})

with open('ADA_SMOTE_Cov.csv', 'w') as file:
    pred_ada.to_csv(file, index = False, header = True)

files.download('ADA_SMOTE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [70]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_LASSO, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

smote = SMOTE(random_state = 2025)
X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

In [71]:
np.random.seed(2025)
ada_grid_lasso = GridSearchCV(AdaBoostClassifier(random_state = 2025), param_grid = params, cv = split_lasso, n_jobs = -1, scoring = 'f1_macro')
ada_grid_lasso.fit(X_temp, y_temp)
print('Best Parameters:', ada_grid_lasso.best_params_)

Best Parameters: {'estimator': DecisionTreeClassifier(max_depth=2, random_state=2025), 'learning_rate': 1.0, 'n_estimators': 50}


In [72]:
np.random.seed(2025)
ada_model_smote = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 2, random_state = 2025), learning_rate = 1.0, n_estimators = 50, random_state = 2025)
ada_model_smote.fit(X_resampled, y_resampled)
y_pred_smote = ada_model_smote.predict(X_test)

print(classification_report(y_test, y_pred_smote))
y_pred_smote_ada = ada_model_smote.predict(X_LASSO_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [73]:
from google.colab import files

pred_ada = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_smote_ada})

with open('ADA_SMOTE_LASSO_Cov.csv', 'w') as file:
    pred_ada.to_csv(file, index = False, header = True)

files.download('ADA_SMOTE_LASSO_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>